# Nonogram
Ovaj projekt implementira nonogram u Pythonu

## Uvoženje random biblioteke
Funkcija **random** nam pomaže generirati mrežu na način da u svakom redu pomoću append na red listu koristimo random.choice između 0 i 1.
- **0** reprezentira ćeliju koja je prazna
- **1** reprezentira ćeliju koja je puna

Biblioteka **os** nam omogućuje dovaćanje trenutnog direktorija jupyter notebooka, da bi učitali nonograme iz datoteke 'nonogrami.txt'.

In [10]:
import random
import os
import itertools

## Funkcija za učitavanje nonograma
Učitava nonograme iz zadane datoteke.

**Datoteka** mora imati sljedeći format, komentare ignoriramo, brojxbroj označava početak nonograma:
```plaintext
# Nonogram 1
5x5
1 0 0 0 1
0 1 1 1 0
1 1 0 0 1
0 1 0 1 0
0 0 0 1 0

# Nonogram 2
4x4
1 0 0 0
0 1 1 1
1 1 0 0
0 1 0 1

In [11]:
def ucitaj_nonograme(putanja_datoteke):
    # lista koja sadrži sve nonograme
    nonogrami = []
    # sadrži trenutni nonogram koji obrađujemo
    trenutni_nonogram = []
    # Otvaramo datoteku u modu čitanja
    with open(putanja_datoteke, "r") as datoteka:
        # Obrađujemo svaku liniju u datoteci
        for linija in datoteka:
            # Uklanjamo razmake s lijeve i desne strane ako ih ima
            linija = linija.strip()
            if not linija or linija.startswith("#"):  # Preskačemo prazne linije i komentare
                continue
            if "x" in linija:  # Novi nonogram počinje, npr. "5x5"
                # Trenutni nonogram se sprema samo ako sadrži podatke
                if trenutni_nonogram:
                    # Dodajemo ga u listu nonograma
                    nonogrami.append(trenutni_nonogram)
                    # Praznimo varijablu za učitavanje sljedećeg nonograma
                    trenutni_nonogram = []
            else:
                red = list(map(int, linija.split()))  # Pretvori red u listu brojeva
                trenutni_nonogram.append(red) # Dodaj red u varijablu trenutni_nonogram
        if trenutni_nonogram:  # Dodaj zadnji nonogram
            nonogrami.append(trenutni_nonogram)
    # Vraća listu nonograma
    return nonogrami

## Funkcija za odabiranje nonograma sa liste učitanih nonograma
- Ukoliko nismo prenesli u argumentu broj nonograma, funkcije će koristiti random.choice za izbor nonograma nasumično

In [12]:
def odaberi_nonogram(nonogrami, broj=None):
    # Ukoliko nismo specificirali koji broj nonograma želimo, vraćamo random nonogram
    if broj is None:
        return random.choice(nonogrami)
    else:
        # Inače vraćamo traženi nonogram
        return nonogrami[broj - 1]

## Funkcija za generiranje tragova određene linije
- Koristimo ju u sljedećoj funkciji za generiranje tragova cijelog nonograma.

In [13]:
def generiraj_tragove(linija):
    tragovi = []
    brojac = 0
    # Za svaku ćeliju u datoj liniji
    for celija in linija:
        # Ako je ćelija 1 povisujemo brojač uzastopnih ćelija
        if celija == 1:
            brojac += 1
        # Ako ćelija nije 1, a brojač nam je već povišen,
        # dodajemo broj uzastopnih ćelija na listu i resetiramo brojač
        elif brojac > 0:
            tragovi.append(brojac)
            brojac = 0
    # Provjera završnog brojača, ako na kraju linije još postoji aktivan brojač,
    # što znači da niz 1 nije prekinut, dodajemo ga na listu tragova
    if brojac > 0:
        tragovi.append(brojac)
    # Vraćamo tragove ako lista nije prazna, inače vraćamo [0] što znači da nema
    # ispunjenih ćelija u toj liniji
    return tragovi if tragovi else [0]

## Funkcija za dohvaćanje nonograma i generiranje tragova
- Koristi prijašnje funkcije za učitavanje nonograma.
- Po defaultu izabire prvi nonogram iz datoteke.
Koristi funkciju generiraj_tragove za generiranje tragova za svaki red, kao i za svaku kolumnu.
- Vraća nonogram, tragove redova i tragove kolumni.

In [14]:
def dohvati_nonogram_sa_tragovima(broj=1):
    # Automatski pronađi putanju do trenutnog notebooka
    putanja_do_datoteke = os.path.join(os.getcwd(), 'nonogrami.txt')
    nonogrami = ucitaj_nonograme(str(putanja_do_datoteke))

    mreza = odaberi_nonogram(nonogrami, broj)
    # Koristimo list comprehension za izračun tragova za svaki red mreže
    tragovi_redova = [generiraj_tragove(red) for red in mreza]
    '''
      Kako funkcionira zip(*mreza):
      Uzima prvi element svakog reda i od toga stvara prvi stupac,
      i tako dalje.
      Na primjer:
      Redovi:       Stupci:
        [1, 0, 0, 1, 1] -> (1, 0, 1, 1, 0)
        [0, 1, 1, 0, 0] -> (0, 1, 0, 1, 0)
        [1, 0, 0, 1, 0] -> (0, 1, 0, 1, 0)
        [1, 1, 1, 0, 0] -> (1, 0, 1, 0, 1)
        [0, 0, 0, 1, 1] -> (1, 0, 0, 0, 1)

      Nakon što smo dobili takav rezultat, svaki red proslijeđujemo funkciji
      generiraj_tragove koja vraća tragove za taj red, i kombiniramo sve u
      novu listu.
    '''
    tragovi_kolumni = [generiraj_tragove(kolumna) for kolumna in zip(*mreza)]

    return mreza, tragovi_redova, tragovi_kolumni

## Funkcija za prikaz (ispis) nonograma
- Ovoj funkciji moramo prenijeti mrežu, tragove redova i kolumni da bi ispisali nonogram.

In [15]:
def prikaz_nonograma(mreza, tragovi_redova, tragovi_kolumni):
    # Pretpostavljamo da je isti broj redova i stupaca, tj kvadratna mreža
    velicina = len(mreza)
    # Maksimalan broj tragova u redovima, koristimo za poravnanje prikaza redova
    maks_tragovi_redova = max(len(trag) for trag in tragovi_redova)
    # Maksimalni broj tragova u stupcima, koristimo na vrhu stupaca
    maks_tragovi_kolumni = max(len(trag) for trag in tragovi_kolumni)

    # Ispis brojeva kolumni (stupaca)
    for i in range(maks_tragovi_kolumni):
        # Inicijalni razmak za poravnanje s kolumnama
        linija = " " * (maks_tragovi_redova * 2 + 3)  # Prostor za tragove redova + razmak za poravnanje
        for kolumna in tragovi_kolumni:
            if i < maks_tragovi_kolumni - len(kolumna):  # Ako je redak iznad dostupnih tragova
                linija += "  "  # Prazan prostor
            else:
                linija += f"{kolumna[i - (maks_tragovi_kolumni - len(kolumna))]} "
        print(linija)

    # Ispis redova s brojevima
    for red, tragovi in zip(mreza, tragovi_redova):
        trag_str = " ".join(map(str, tragovi)).rjust(maks_tragovi_redova * 2)
        print(trag_str + " | " + " ".join(["█" if celija == 1 else "." for celija in red]))

## Funkcije za popunu nonograma

In [16]:
def potpuna_popuna(mreza, tragovi, stupci=False):
    if stupci:
        # Transponiraj mrežu za rad sa stupcima
        mreza = [list(red) for red in zip(*mreza)]

    for i, trag in enumerate(tragovi):
        # Provjera: Ako trag sadrži samo jedan broj i jednak je dužini mreže/stupca
        if len(trag) == 1 and trag[0] == len(mreza[i]):
            n = trag[0]
            pozicija = 0
            for _ in range(n):
                mreza[i][pozicija] = True  # Oznaka popunjenog kvadrata
                pozicija += 1

    if stupci:
        # Vratimo mrežu u izvorni oblik nakon obrade stupaca
        mreza = [list(red) for red in zip(*mreza)]

    return mreza

# Strategija segmentacija - redovi
def segmentacija_redovi(mreza, tragovi):
    for i, trag in enumerate(tragovi):
        ukupno = sum(trag) + len(trag) - 1  # Suma tragova + minimalni razmak
        if ukupno == len(mreza[i]):  # Ako je suma jednaka dužini mreže
            pozicija = 0
            for n in trag:
                # Popunjavanje kvadrata za trenutni segment
                for _ in range(n):
                    if not mreza[i][pozicija]:  # Provjeravamo je li pozicija već označena kao True
                        mreza[i][pozicija] = True  # Oznaka popunjenog kvadrata
                    pozicija += 1
                # Ako nije posljednji segment, dodaj razmak
                if pozicija < len(mreza[i]):
                    mreza[i][pozicija] = "."  # Oznaka razmaka
                    pozicija += 1
    return mreza

# Strategija segmentacija - stupci
def segmentacija_stupci(mreza, tragovi):
    transponirana_mreza = [list(red) for red in zip(*mreza)]  # Transponiraj mrežu za obradu stupaca
    for i, trag in enumerate(tragovi):
        ukupno = sum(trag) + len(trag) - 1  # Suma tragova + minimalni razmak
        if ukupno == len(transponirana_mreza[i]):  # Ako je suma jednaka dužini stupca
            pozicija = 0
            for n in trag:
                # Popunjavanje kvadrata za trenutni segment
                for _ in range(n):
                    if not transponirana_mreza[i][pozicija]:  # Provjeravamo je li pozicija već označena kao True
                        transponirana_mreza[i][pozicija] = True  # Oznaka popunjenog kvadrata
                    pozicija += 1
                # Ako nije posljednji segment, dodaj razmak
                if pozicija < len(transponirana_mreza[i]):
                    transponirana_mreza[i][pozicija] = "."  # Oznaka razmaka
                    pozicija += 1
    # Vratimo transponiranu mrežu natrag u originalnu formu
    mreza = [list(red) for red in zip(*transponirana_mreza)]  # Transponiraj natrag
    return mreza


# Strategija identifikacija zajamčenih ispunjenja
def generiraj_kombinacije(duljina, trag):
    """Generira sve moguće kombinacije za trag u mreži, uključujući minimalno jedno prazno polje između segmenata."""
    if len(trag) == 1:
        # Generiraj kombinacije za trag s jednim brojem
        broj_praznina = duljina - trag[0]
        if broj_praznina < 0:
            return []  # Nema rješenja ako trag ne stane u mrežu

        kombinacije = []
        for raspodjela in itertools.combinations(range(duljina - trag[0] + 1), 1):
            mreza = [False] * duljina
            for i in raspodjela:
                for j in range(trag[0]):
                    mreza[i + j] = True
            kombinacije.append(mreza)
        return kombinacije

    else:
        # Generiraj kombinacije za trag s više brojeva
        ukupna_duljina = sum(trag) + (len(trag) - 1)

        if ukupna_duljina > duljina:
            return []  # Nema dovoljno mjesta za sve segmente

        kombinacije = []

        # Provjeravamo da li ukupna duljina segmenata i praznina odgovara duljini mreže
        if ukupna_duljina == duljina:
            # Ako je ukupna duljina jednaka duljini mreže, postoji samo jedna kombinacija
            mreza = [False] * duljina
            pozicija = 0
            for i, trag_segment in enumerate(trag):
                for j in range(trag_segment):
                    mreza[pozicija] = True
                    pozicija += 1
                if i < len(trag) - 1:
                    pozicija += 1  # Dodajemo prazninu između segmenata
            kombinacije.append(mreza)
        else:
            # Prva kombinacija: segmenti na početku sa praznim prostorom između
            pozicija = 0
            mreza = [False] * duljina
            for i, trag_segment in enumerate(trag):
                for j in range(trag_segment):
                    mreza[pozicija] = True
                    pozicija += 1
                if i < len(trag) - 1:
                    pozicija += 1  # Dodajemo prazninu između segmenata
            kombinacije.append(mreza)

            # Druga kombinacija: segmenti na početku sa dva prazna mjesta između
            pozicija = 0
            mreza = [False] * duljina
            for i, trag_segment in enumerate(trag):
                for j in range(trag_segment):
                    mreza[pozicija] = True
                    pozicija += 1
                if i < len(trag) - 1:
                    pozicija += 2  # Dodajemo dva prazna mjesta između segmenata
            kombinacije.append(mreza)

            # Treća kombinacija: prvi segment pomičemo na drugo mjesto
            pozicija = 1  # Prvo pomičemo prvi segment na drugo mjesto
            mreza = [False] * duljina
            for i, trag_segment in enumerate(trag):
                for j in range(trag_segment):
                    mreza[pozicija] = True
                    pozicija += 1
                if i < len(trag) - 1:
                    pozicija += 1  # Dodajemo prazninu između segmenata
            kombinacije.append(mreza)

    return kombinacije

def identifikacija_zajamcenih_redovi(mreza, tragovi):
    duljina = len(mreza[0])

    # Iteriraj kroz svaki trag u tragovima
    for i, trag in enumerate(tragovi):
        # Generiraj sve moguće kombinacije za trenutni trag
        sve_kombinacije = list(generiraj_kombinacije(duljina, trag))

        if sve_kombinacije:
            # Provodimo preklapanje svih kombinacija (presjek)
            preklapanje = [all(k[j] for k in sve_kombinacije) for j in range(duljina)]

            # Ažuriraj mrežu samo za sigurno popunjena polja
            for j in range(duljina):
                if preklapanje[j]:
                    mreza[i][j] = True  # Označavamo kao sigurno popunjeno

    return mreza

def identifikacija_zajamcenih_stupci(mreza, tragovi_stupci):
    duljina = len(mreza)
    broj_stupaca = len(mreza[0])

    # Iteriraj kroz svaki trag u tragovima_stupci (za stupce)
    for j in range(broj_stupaca):
        # Izvucimo tragove za trenutni stupac
        trag = tragovi_stupci[j]  # Ovde uzimamo tragove iz tragovi_stupci za stupce

        # Generiraj sve moguće kombinacije za trenutni stupac
        sve_kombinacije = list(generiraj_kombinacije(duljina, trag))

        if sve_kombinacije:
            # Provodimo preklapanje svih kombinacija (presjek)
            preklapanje = [all(k[i] for k in sve_kombinacije) for i in range(duljina)]

            # Ažuriraj mrežu samo za sigurno popunjena polja
            for i in range(duljina):
                if preklapanje[i]:
                    mreza[i][j] = True  # Označavamo kao sigurno popunjeno

    return mreza

# Strategija označavanje praznih kvadrata
def oznacavanje_praznih(mreza, tragovi_redovi, tragovi_stupci):
    # Funkcija za označavanje praznih polja u redovima
    def obradi_redove(mreza, tragovi):
        for i, trag in enumerate(tragovi):
            true_pozicije = [j for j, val in enumerate(mreza[i]) if val is True]

            # Kada trag sadrži samo jedan broj
            if len(trag) == 1:
                # Ako suma True polja nije jednaka sumi traga, preskoči obradu
                if len(true_pozicije) != trag[0]:
                    continue

                # Označavamo prazna mjesta prije i nakon True segmenta
                if true_pozicije:
                    if true_pozicije[0] > 0 and mreza[i][true_pozicije[0] - 1] is False:
                        mreza[i][true_pozicije[0] - 1] = '.'
                    if true_pozicije[-1] < len(mreza[i]) - 1 and mreza[i][true_pozicije[-1] + 1] is False:
                        mreza[i][true_pozicije[-1] + 1] = '.'

                # Preostala mjesta označavamo praznim ako ne mogu pripadati segmentu
                prvi_moguci = max(0, true_pozicije[0] - (trag[0] - len(true_pozicije)))
                zadnji_moguci = min(len(mreza[i]), true_pozicije[-1] + (trag[0] - len(true_pozicije)) + 1)
                for j in range(len(mreza[i])):
                    if mreza[i][j] is False and (j < prvi_moguci or j >= zadnji_moguci):
                        mreza[i][j] = '.'

            # Kada trag sadrži više brojeva (ne mijenja se)
            elif len(trag) > 1:
                for j in true_pozicije:
                    if j > 0 and mreza[i][j - 1] is False:
                        mreza[i][j - 1] = '.'
                    if j < len(mreza[i]) - 1 and mreza[i][j + 1] is False:
                        mreza[i][j + 1] = '.'
                for k in range(len(true_pozicije) - 1):
                    if true_pozicije[k + 1] - true_pozicije[k] > 1:
                        for j in range(true_pozicije[k] + 1, true_pozicije[k + 1]):
                            if mreza[i][j] is False:
                                mreza[i][j] = '.'

        return mreza

    # Funkcija za označavanje praznih polja u stupcima
    def obradi_stupce(mreza, tragovi):
        transponirana_mreza = [list(red) for red in zip(*mreza)]
        transponirana_mreza = obradi_redove(transponirana_mreza, tragovi)
        return [list(red) for red in zip(*transponirana_mreza)]

    # Obradimo redove
    mreza = obradi_redove(mreza, tragovi_redovi)

    # Obradimo stupce
    mreza = obradi_stupce(mreza, tragovi_stupci)

    return mreza

# Strategija provjere granica
def provjera_granica(mreza, tragovi_redovi, tragovi_stupci):
    def popuni_granice_red(mreza, i, trag):
        red = mreza[i]
        duljina_reda = len(red)
        true_pozicije = [j for j, val in enumerate(red) if val is True]
        false_pozicije = [j for j, val in enumerate(red) if val == '.']

        if not true_pozicije:
            return red  # Ako nema True polja, ne radimo ništa

        p_start, p_end = min(true_pozicije), max(true_pozicije)

        # Ako trag ima samo jedan segment
        if len(trag) == 1:
            # Spoji True polja ako nisu uzastopna i poštuj False granice
            for j in range(p_start, p_start + trag[0]):
                if j < duljina_reda and red[j] != '.':
                    red[j] = True

            # Označi prazna polja izvan granica segmenta
            for j in range(0, p_start):
                if red[j] != True:
                    red[j] = '.'

            for j in range(p_start + trag[0], duljina_reda):
                if red[j] != True:
                    red[j] = '.'

        # Ako trag ima više segmenata, ne spajamo True polja
        else:
            for j in range(0, p_start):
                if red[j] != True:
                    red[j] = '.'
            for j in range(p_end + 1, duljina_reda):
                if red[j] != True:
                    red[j] = '.'

        # Ako broj True polja ne odgovara broju segmenata u tragu, označi s '?'
        if len(true_pozicije) != sum(trag):
            for j in range(duljina_reda):
                if red[j] != True:
                    red[j] = '?'

        return red

    def popuni_granice_stupac(mreza, j, trag):
        stupac = [red[j] for red in mreza]
        duljina_stupca = len(stupac)
        true_pozicije = [i for i, val in enumerate(stupac) if val is True]
        false_pozicije = [i for i, val in enumerate(stupac) if val == '.']

        if not true_pozicije:
            return stupac  # Ako nema True polja, ne radimo ništa

        p_start, p_end = min(true_pozicije), max(true_pozicije)

        # Ako trag ima samo jedan segment
        if len(trag) == 1:
            # Spoji True polja ako nisu uzastopna i poštuj False granice
            for i in range(p_start, p_start + trag[0]):
                if i < duljina_stupca and stupac[i] != '.':
                    stupac[i] = True

            # Označi prazna polja izvan granica segmenta
            for i in range(0, p_start):
                if stupac[i] != True:
                    stupac[i] = '.'
            for i in range(p_start + trag[0], duljina_stupca):
                if stupac[i] != True:
                    stupac[i] = '.'

        # Ako trag ima više segmenata, ne spajamo True polja
        else:
            for i in range(0, p_start):
                if stupac[i] != True:
                    stupac[i] = '.'
            for i in range(p_end + 1, duljina_stupca):
                if stupac[i] != True:
                    stupac[i] = '.'

        # Ako broj True polja ne odgovara broju segmenata u tragu, označi s '?'
        if len(true_pozicije) != sum(trag):
            for i in range(duljina_stupca):
                if stupac[i] != True:
                    stupac[i] = '?'

        # Ažuriraj mrežu stupca s novim vrijednostima
        for i, val in enumerate(stupac):
            if mreza[i][j] != '.':
                mreza[i][j] = val

    def prenesi_informacije(mreza, tragovi_redovi, tragovi_stupci):
        # Kada je red popunjen, ažuriraj stupce
        for i, red in enumerate(mreza):
            if all(val in [True, '.'] for val in red):  # Red je kompletno riješen
                for j, val in enumerate(red):
                    if val is True:
                        trag = tragovi_stupci[j]
                        popuni_granice_stupac(mreza, j, trag)

        # Kada je stupac popunjen, ažuriraj redove
        for j in range(len(mreza[0])):
            stupac = [red[j] for red in mreza]
            if all(val in [True, '.'] for val in stupac):  # Stupac je kompletno riješen
                for i, val in enumerate(stupac):
                    if val is True:
                        trag = tragovi_redovi[i]
                        mreza[i] = popuni_granice_red(mreza, i, tragovi_redovi[i])

    # Obradi redove
    for i, trag in enumerate(tragovi_redovi):
        mreza[i] = popuni_granice_red(mreza, i, trag)

    # Obradi stupce
    for j, trag in enumerate(tragovi_stupci):
        popuni_granice_stupac(mreza, j, trag)

    # Prenesi informacije između redaka i stupaca
    prenesi_informacije(mreza, tragovi_redovi, tragovi_stupci)

    return mreza


# Strategija kombiniranja informacija
def kombiniraj_informacije(mreza, tragovi_redovi, tragovi_stupci):
    def obradi_red(mreza, i, trag):
        red = mreza[i]
        true_count = red.count(True)
        nepoznata_pozicije = [j for j, val in enumerate(red) if val == '?']

        # Ako broj True polja odgovara tragu, označi sva nepoznata polja kao prazna ('.')
        if true_count == sum(trag):
            for j in nepoznata_pozicije:
                red[j] = '.'

        # Ako broj True polja je manji od traga, popuni preostala nepoznata polja s True
        elif true_count < sum(trag):
            for j in nepoznata_pozicije:
                if true_count < sum(trag):
                    red[j] = True
                    true_count += 1

        return red

    def obradi_stupac(mreza, j, trag):
        stupac = [red[j] for red in mreza]
        true_count = stupac.count(True)
        nepoznata_pozicije = [i for i, val in enumerate(stupac) if val == '?']

        # Ako broj True polja odgovara tragu, označi sva nepoznata polja kao prazna ('.')
        if true_count == sum(trag):
            for i in nepoznata_pozicije:
                mreza[i][j] = '.'

        # Ako broj True polja je manji od traga, popuni preostala nepoznata polja s True
        elif true_count < sum(trag):
            for i in nepoznata_pozicije:
                if true_count < sum(trag):
                    mreza[i][j] = True
                    true_count += 1

    # Obrada redova
    for i, trag in enumerate(tragovi_redovi):
        mreza[i] = obradi_red(mreza, i, trag)

    # Obrada stupaca
    for j, trag in enumerate(tragovi_stupci):
        obradi_stupac(mreza, j, trag)

    return mreza


## Funkcija koja riješava nonogram kombinirajući sve ostale funkcije

In [17]:
def rijesi_nonogram(mreza, tragovi_redova, tragovi_kolumni):


    # Inicijalizacija prazne mreže
    velicina = len(tragovi_redova)
    mreza2 = [[False for _ in range(velicina)] for _ in range(velicina)]

    print("Funkcija: potpuna_popuna(mreza2, tragovi_redova)\n")
    mreza2 = potpuna_popuna(mreza2, tragovi_redova)
    prikaz_nonograma(mreza2, tragovi_redova, tragovi_kolumni)

    print("\nFunkcija: potpuna_popuna(mreza2, tragovi_kolumni, True)\n")
    mreza2 = potpuna_popuna(mreza2, tragovi_kolumni, True)
    prikaz_nonograma(mreza2, tragovi_redova, tragovi_kolumni)

    print("\nFunkcija: segmentacija_redovi(mreza, tragovi_redova)\n")
    mreza2 = segmentacija_redovi(mreza2, tragovi_redova)
    prikaz_nonograma(mreza2, tragovi_redova, tragovi_kolumni)

    print("\nFunkcija: segmentacija_stupci(mreza, tragovi_kolumni)\n")
    mreza2 = segmentacija_stupci(mreza2, tragovi_kolumni)
    prikaz_nonograma(mreza2, tragovi_redova, tragovi_kolumni)

    print("\nFunkcija: identifikacija_zajamcenih_redovi(mreza, tragovi_redova)\n")
    mreza2 = identifikacija_zajamcenih_redovi(mreza2, tragovi_redova)
    prikaz_nonograma(mreza2, tragovi_redova, tragovi_kolumni)

    print("\nFunkcija: identifikacija_zajamcenih_stupci(mreza, tragovi_kolumni)\n")
    mreza2 = identifikacija_zajamcenih_stupci(mreza2, tragovi_kolumni)
    prikaz_nonograma(mreza2, tragovi_redova, tragovi_kolumni)

    print("\nFunkcija: oznacavanje_praznih(mreza, tragovi_redova, tragovi_kolumni)\n")
    mreza2 = oznacavanje_praznih(mreza2, tragovi_redova, tragovi_kolumni)
    prikaz_nonograma(mreza2, tragovi_redova, tragovi_kolumni)

    print("\nFunkcija: provjera_granica(mreza, tragovi_redova, tragovi_kolumni)\n")
    mreza2 = provjera_granica(mreza2, tragovi_redova, tragovi_kolumni)
    prikaz_nonograma(mreza2, tragovi_redova, tragovi_kolumni)

    print("\nFunkcija: kombiniraj_informacije(mreza, tragovi_redova, tragovi_kolumni)\n")
    mreza2 = kombiniraj_informacije(mreza2, tragovi_redova, tragovi_kolumni)
    prikaz_nonograma(mreza2, tragovi_redova, tragovi_kolumni)

## Glavni dio programa

In [18]:
mreza, tragovi_redova, tragovi_kolumni = dohvati_nonogram_sa_tragovima(1)

print("Odabrani nonogram:")
prikaz_nonograma(mreza, tragovi_redova, tragovi_kolumni)

# Rješavanje nonograma

# Ispis rezultata
print("\nRješenje:\n")
rijesi_nonogram(mreza, tragovi_redova, tragovi_kolumni)

Odabrani nonogram:
         2 2 1 2   
         1 1 2 1 5 
   1 1 | █ . . . █
     5 | █ █ █ █ █
   1 2 | . █ . █ █
 1 1 1 | █ . █ . █
     4 | . █ █ █ █

Rješenje:

Funkcija: potpuna_popuna(mreza2, tragovi_redova)

         2 2 1 2   
         1 1 2 1 5 
   1 1 | . . . . .
     5 | █ █ █ █ █
   1 2 | . . . . .
 1 1 1 | . . . . .
     4 | . . . . .

Funkcija: potpuna_popuna(mreza2, tragovi_kolumni, True)

         2 2 1 2   
         1 1 2 1 5 
   1 1 | . . . . █
     5 | █ █ █ █ █
   1 2 | . . . . █
 1 1 1 | . . . . █
     4 | . . . . █

Funkcija: segmentacija_redovi(mreza, tragovi_redova)

         2 2 1 2   
         1 1 2 1 5 
   1 1 | . . . . █
     5 | █ █ █ █ █
   1 2 | . . . . █
 1 1 1 | █ . █ . █
     4 | . . . . █

Funkcija: segmentacija_stupci(mreza, tragovi_kolumni)

         2 2 1 2   
         1 1 2 1 5 
   1 1 | . . . . █
     5 | █ █ █ █ █
   1 2 | . . . . █
 1 1 1 | █ . █ . █
     4 | . . . . █

Funkcija: identifikacija_zajamcenih_redovi(mreza, tragovi_redova)

       